In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit


### Install adk

In [1]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.

In [2]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[1]
print(MODEL_NAME)

gemini-2.5-pro-preview-03-25


In [3]:
# @title Authentication to access to GCP
import sys
from IPython.display import display, Markdown

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Agent on adk

In [4]:
#@title Build a Agent with adk.
from google.adk.agents import Agent
from google.adk.tools import google_search

agent = Agent(
    name="search_assistant",
    model=MODEL_NAME,
    instruction="You are a helpful assistant. Answer user questions using Google Search when needed.",
    description="An assistant that can search the web.",
    tools=[google_search] # Use Google search tool
)


In [5]:
#@title Integrate an agent with Agent Engine

from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(agent=agent)

for event in app.stream_query(
   user_id="shins",  # Required
   message="What is the exchange rate from US dollars to Korean currency today ?",
):
  print(event)
  outcome = event['content']['parts'][0]['text']
  display(Markdown(outcome))

  #print(event['content']['parts'][0]['text'])

{'content': {'parts': [{'text': "Based on the search results for today, April 30, 2025, the exchange rate from US Dollars (USD) to South Korean Won (KRW) is approximately:\n\n*   **1 USD ≈ 1,431.7 KRW**\n*   **1 USD ≈ 1,432.13 KRW** (as of April 29)\n*   **1 USD ≈ 1,432.67 KRW**\n*   **1 USD ≈ 1,432.75 KRW**\n*   **1 USD ≈ 1,433.33 KRW**\n\n**Key Points:**\n\n*   **Fluctuating Rates:** Exchange rates change constantly due to market dynamics. The exact rate you get will depend on when and where you make the exchange.\n*   **Provider Differences:** Different banks or money transfer services (like Revolut, Wise, Western Union, XE) may offer slightly different rates.\n*   **Fees:** Be aware that fees may apply when converting currencies, which can affect the final amount you receive. Some providers might build fees into the exchange rate itself.\n\nFor the most accurate and current rate for a specific transaction, it's best to check directly with the bank or currency exchange service you p

Based on the search results for today, April 30, 2025, the exchange rate from US Dollars (USD) to South Korean Won (KRW) is approximately:

*   **1 USD ≈ 1,431.7 KRW**
*   **1 USD ≈ 1,432.13 KRW** (as of April 29)
*   **1 USD ≈ 1,432.67 KRW**
*   **1 USD ≈ 1,432.75 KRW**
*   **1 USD ≈ 1,433.33 KRW**

**Key Points:**

*   **Fluctuating Rates:** Exchange rates change constantly due to market dynamics. The exact rate you get will depend on when and where you make the exchange.
*   **Provider Differences:** Different banks or money transfer services (like Revolut, Wise, Western Union, XE) may offer slightly different rates.
*   **Fees:** Be aware that fees may apply when converting currencies, which can affect the final amount you receive. Some providers might build fees into the exchange rate itself.

For the most accurate and current rate for a specific transaction, it's best to check directly with the bank or currency exchange service you plan to use.

## Deploy agent on Agent Engine for a managed service.

In [6]:
#@title Create a bucket to store agent engine artifacts

BUCKET_URI = f"gs://agent-0417"
!gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0417/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0417' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [8]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [9]:
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

In [10]:
#@title Initialize Vertex AI with Staging Bucket.
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [11]:
# @title Deploy your agent on Vertex AI

from vertexai import agent_engines

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_create

remote_agent = agent_engines.create(
    app,
    display_name="currency agent with adk",
    gcs_dir_name = "ai-agent-adk",
    description="This is a simple agent with adk on agent engine.",
    requirements=[
        "google-adk",
        "cloudpickle==3.0",
    ],
    extra_packages = []
)

INFO:vertexai.agent_engines:Identified the following requirements: {'cloudpickle': '3.1.1', 'pydantic': '2.10.6', 'google-cloud-aiplatform': '1.90.0'}
INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.10.6'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-adk', 'cloudpickle==3.0', 'pydantic==2.10.6']
INFO:vertexai.agent_engines:Using bucket agent-0417
INFO:vertexai.agent_engines:Wrote to gs://agent-0417/ai-agent-adk/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-adk/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-adk/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/8478105463362158592/operations/5743481059941548032
INFO:vertexai.agent_engines:View progress 

In [13]:
# @title Query from remote engine.

for event in remote_agent.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on today?",
):
  outcome = event['content']['parts'][0]['text']
  display(Markdown(outcome))

As of Wednesday, April 30, 2025, the exchange rate between US Dollars (USD) and South Korean Won (KRW) is approximately:

*   **1 USD ≈ 1,431 - 1,434 KRW**

Here are some specific rates found from different sources around this time:

*   XE.com: 1 USD = 1,431.7 KRW
*   Currency.ME.UK: 1 USD = 1432.1513 KRW (as of 00:08, April 30, 2025)
*   Revolut: 1 USD = 1,432.75 KRW / 1 USD = 1,432.67 KRW
*   Wise: 1 USD = 1,433.33 KRW
*   X-Rates: 1 USD = 1,432.31 KRW (as of April 29, 2025, 20:34 UTC)
*   Trading Economics: 1 USD = 1,432.13 KRW (as of April 29, 2025)

**Example Conversions (using approx. 1 USD = 1,432 KRW):**

*   10 USD ≈ 14,320 KRW
*   50 USD ≈ 71,600 KRW
*   100 USD ≈ 143,200 KRW
*   1,000 USD ≈ 1,432,000 KRW

**Important Considerations:**

*   **Fluctuating Rates:** Exchange rates change constantly throughout the day based on market trading.
*   **Transaction Rates vs. Mid-Market Rates:** The rates listed above are generally "mid-market" rates (the midpoint between buy and sell rates). When you actually exchange money (e.g., through a bank, currency exchange service, or money transfer provider like Wise, Revolut, Western Union, OFX), you will likely receive a slightly different rate due to transaction fees or the provider's specific customer rate. Always check the rate offered by the specific service you plan to use.

In [14]:
#@title Download common functions and execute.
!wget https://raw.githubusercontent.com/shins777/ai_agent/main/common/common_agent_engine.ipynb

# Execute common function for agent engin.
%run "common_agent_engine.ipynb"

--2025-04-30 01:55:24--  https://raw.githubusercontent.com/shins777/ai_agent/main/common/common_agent_engine.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6863 (6.7K) [text/plain]
Saving to: ‘common_agent_engine.ipynb’

common_agent_engine 100%[===================>]   6.70K  --.-KB/s    in 0s      

2025-04-30 01:55:24 (60.8 MB/s) - ‘common_agent_engine.ipynb’ saved [6863/6863]

Function get_agent_engine(display_name:str) added
Function show_agents() added
Function delete_agent(name:str)


## Agent Management

In [20]:
#@title Show agent list
show_agents()

Agent 0: 
	Display Name [currency agent with adk] 
	Name [8478105463362158592] 
	Creation Time [2025-04-30 01:48:23.293469+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/8478105463362158592]

Agent 1: 
	Display Name [currency agent with adk] 
	Name [2135700181885124608] 
	Creation Time [2025-04-26 14:53:02.246908+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/2135700181885124608]

Agent 2: 
	Display Name [currency agent with adk] 
	Name [8720807262030921728] 
	Creation Time [2025-04-18 08:10:36.636926+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/8720807262030921728]



In [16]:
#@title Get a agent with display name.
agent = get_agent_engine(display_name="currency agent with adk")
agent

resource name: projects/ai-hangsik/locations/us-central1/reasoningEngines/8478105463362158592

In [18]:
# @title Query from remote engine.
for event in agent.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on toady?",
):
  outcome = event['content']['parts'][0]['text']
  display(Markdown(outcome))

As of Wednesday, April 30, 2025, the exchange rate between US Dollars (USD) and South Korean Won (KRW) is approximately:

*   **1 USD = 1,432.13 KRW** (Based on the previous trading session ending April 29th)
*   **1 USD = 1,431.7 KRW**
*   **1 USD = 1,432.67 KRW**
*   **1 USD = 1,433.33 KRW**

Keep in mind that:

1.  **Rates Fluctuate:** Exchange rates change constantly throughout the day.
2.  **Provider Differences:** Different currency exchange services (like banks, Wise, XE, Revolut, Western Union) may offer slightly different rates due to fees or markups. The rates shown are often the "mid-market" or "spot" rate, which is the rate traded between banks.
3.  **Spot vs. Forward Rates:** The rates above generally refer to the spot rate, meaning the rate for immediate exchange.

For the most precise rate at the exact moment you plan to exchange currency, it's best to check a live currency converter or the specific provider you intend to use.

In [ ]:
#@title Delete a agent with name.
delete_agent(name="8720807262030921728")

## End of notebook